In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from datetime import datetime, timedelta

In [3]:
xl_data  = pd.read_excel("IE01.552.051-data_pack.xlsx",sheet_name="Sensor_Data")
xl_data.shape

(109009, 49)

In [4]:
failure = pd.read_excel("IE01.552.051-data_pack.xlsx",sheet_name="Failure_Data")
failure.shape

(25, 11)

In [5]:
# Standardize the start and end time of failures. 
failure["FailureStartDate"] = failure["FailureStartDate"].apply(lambda x:datetime.strptime(x,"%d.%m.%Y").date())
failure["FailureStartTime"] = failure["FailureStartTime"].apply(lambda x:datetime.strptime(x,"%H:%M:%S").time())
failure["Fstart"] = [datetime.combine(failure.iloc[k]["FailureStartDate"],failure.iloc[k]["FailureStartTime"]) for k in range(len(failure))]
failure["Fend"] = [failure.iloc[k]["Fstart"]+timedelta(hours=failure.iloc[k]["actual work\nin hours"]) for k in range(len(failure))]

# Setting type of failure. 1 - Non-maintenance ; 2 - Deferred Maintenance ; 3 - Immediate Maintenance 
failure["Type"] = failure["Order Type"].apply(lambda x:int(x[-1])+1)

In [6]:
# Construct a list of "timestamps" when there was an observed failure.
failure_intervals = pd.Series()
for k in range(len(failure)):
    s = failure.iloc[k]["Fstart"].replace(second=0)
    while s <= failure.iloc[k]["Fend"]:
        failure_intervals[s] = failure.iloc[k]["Type"]
        s+=timedelta(minutes=1)   #make it per minute to have higher granuarity

In [7]:
def failtype(ts):
    if ts in failure_intervals.index:
        return failure_intervals[ts]
    else:
        return 0

sandbox = xl_data.copy()
sandbox["labels"] = sandbox["Timestamp"].apply(failtype)
sandbox.to_csv("easy_load.csv")
print(Counter(sandbox["labels"].values))

Counter({0: 108440, 2: 424, 1: 145})


In [54]:
sandbox.set_index(["Timestamp"],drop=True,inplace=True)
sandbox.sort_index(inplace=True)

features = list(sandbox.columns)
features.remove("labels")
cat_cols = ["AI552051.754_ALM","BACT.552051","BCMPLT.552051","FAL552051.754","HV552051.331","HV552051.332","LAH552051.670","LAH552051.678","LAH552051.680","M552051.801","M552051.802","M552051.823","M552051.826","M552051.871","MAINT.552051","MODMAN.552051","PARTREC.552051","QCA552051_001","RUN.552051","SIC552051.801_ALM","SSOALM.552051","VI552051.748_ALM","ZS552051.737","ZS552051.740","ZS552051_753"]

In [55]:
for col in cat_cols:
    sandbox[col] = sandbox[col].astype("category")
    #print(col, Counter(sandbox[col].to_list()))
    sandbox = pd.get_dummies(sandbox,prefix=[col],columns=[col])
    #sandbox.drop([col],inplace=True,axis=1)
    
sandbox.shape

(109009, 110)

In [56]:
non_cat = list(set(features)-set(cat_cols))
print(non_cat)

['WI552051.675', 'FQI552051.718', 'VI552051.748', 'WI552051.674', 'TIC552051.616', 'PI552051.645', 'AI552051.754', 'MODSTS.552051', 'FIC552051_718', 'CYCLE.552051', 'SIC552051.801_SP', 'PI552051.642', 'SI552051.801', 'PI552051.651', 'PHASE.552051', 'SIA552051.801', 'FI552051_718', 'PIN.552051', 'SIC552051.801_CO', 'PI552051.646', 'PHTIME.552051', 'SIC552051.801', 'PI552051.641']


In [57]:
for col in non_cat:
    sandbox[col] = pd.to_numeric(sandbox[col],errors="coerce").fillna(sandbox[col])
    sandbox[col+"_Bad Input"] = [1 if k == "Bad Input" else 0 for k in sandbox[col].tolist()]
    sandbox[col+"_I/O Timeout"] = [1 if k == "I/O Timeout" else 0 for k in sandbox[col].tolist()]
    #print(col,sandbox[col].dtype)
sandbox[non_cat] = sandbox[non_cat].apply(pd.to_numeric, errors='coerce').fillna(0)

sandbox.shape

(109009, 156)

In [41]:
badinputs = []
no_io = []
for col in sandbox.columns:
    if "Bad Input" in col:
        badinputs.append(col)
    if "I/O Timeout" in col:
        no_io.append(col)

truess = np.zeros(len(sandbox))
trs = np.zeros(len(sandbox))
for col in badinputs:
    truess = truess + sandbox[col]
for col in no_io:
    trs = trs + sandbox[col]

print("Total bad inputs",sandbox.iloc[np.where(truess > 1)].shape)
print("Total i/o timeouts",sandbox.iloc[np.where(trs > 1)].shape)

Total bad inputs (9473, 156)
Total i/o timeouts (1, 156)


In [58]:
feat_cols = list(sandbox.columns)
feat_cols.remove("labels")

In [59]:

marked_index = datetime.strptime("2016-07-01 00:00:00","%Y-%m-%d %H:%M:%S")

train_data = sandbox.loc[:marked_index][feat_cols]
test_data = sandbox.loc[marked_index:][feat_cols]
train_labels = sandbox.loc[:marked_index]["labels"]
test_labels = sandbox.loc[marked_index:]["labels"]
print(train_data.shape,train_labels.shape)
print(test_data.shape,test_labels.shape)



(65809, 155) (65809,)
(43201, 155) (43201,)


In [60]:
train_data.to_csv("data/train_data.csv",index=False)
test_data.to_csv("data/test_data.csv",index=False)
train_labels.to_csv("data/train_labels.csv",index=False)
test_labels.to_csv("data/test_labels.csv",index=False)

/home/venugopal/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/venugopal/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.
